<a href="https://colab.research.google.com/github/gfeyzakorkmaz/Tries/blob/main/Pandas4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
path = '/content/drive/MyDrive/Dataset/complex_marketing_data_enriched.xlsx'

df = pd.read_excel(path)

In [3]:
complex_aggregation = df.groupby('Region').agg(
    Total_Spent=('Total Spent', 'sum'),
    Avg_Discount_Long_Tenure=('Discount Availed', lambda x: x[df['Customer Tenure (Years)'] > 5].mean()),
    Conversion_Rate_High_Spending=('Converted', lambda x: x[df['Total Spent'] > 500].mean())
)
print(complex_aggregation)


        Total_Spent  Avg_Discount_Long_Tenure  Conversion_Rate_High_Spending
Region                                                                      
East           3170                 35.000000                           1.00
North          1940                 37.666667                           0.50
South           750                       NaN                           1.00
West           3060                 42.400000                           0.75


In [4]:
weighted_cart_by_campaign = df[df['Return Rate (%)'] < 5].groupby('Campaign').apply(
    lambda x: (x['Average Cart Value'] * x['Total Spent']).sum() / x['Total Spent'].sum()
)
print(weighted_cart_by_campaign)


Campaign
Black Friday    330.007701
Summer Sale     315.520000
Winter Sale     278.545407
dtype: float64


<ipython-input-4-07eac2f7315a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_cart_by_campaign = df[df['Return Rate (%)'] < 5].groupby('Campaign').apply(


In [5]:
df_sorted = df.sort_values('Age')
df_sorted['Rolling_Avg_Spent'] = df_sorted['Total Spent'].rolling(window=5, min_periods=1).mean()
print(df_sorted[['Age', 'Total Spent', 'Rolling_Avg_Spent']])


    Age  Total Spent  Rolling_Avg_Spent
0    23          150              150.0
11   24          290              220.0
19   25          400              280.0
7    27          120              240.0
15   28          350              262.0
4    29          300              292.0
13   32          320              298.0
9    33          750              368.0
17   35          700              484.0
1    35          600              534.0
14   36          510              576.0
6    38          670              646.0
10   40          610              618.0
5    41          450              568.0
16   42          550              558.0
2    45          800              616.0
12   48          400              562.0
8    50          500              540.0
18   50          250              500.0
3    52          200              430.0


In [6]:
multi_level_analysis = df.groupby(['Region', 'Income Level']).agg(
    Avg_Spent=('Total Spent', 'mean'),
    Std_Spent=('Total Spent', 'std'),
    Customer_Count=('Customer ID', 'count')
)
print(multi_level_analysis)


                      Avg_Spent   Std_Spent  Customer_Count
Region Income Level                                        
East   High          320.000000         NaN               1
       Low           433.333333  162.583312               3
       Medium        310.000000  143.178211               5
North  High          350.000000         NaN               1
       Low           395.000000  388.908730               2
       Medium        800.000000         NaN               1
South  Medium        750.000000         NaN               1
West   High          605.000000  134.350288               2
       Low           600.000000         NaN               1
       Medium        416.666667  152.752523               3


In [7]:
region_spending = df.groupby('Region')['Total Spent'].mean()
high_spending_regions = region_spending[region_spending > 400].index
filtered_customers = df[
    (df['Customer Tenure (Years)'] > 10) &
    (df['Region'].isin(high_spending_regions)) &
    (df['Discount Availed'] > 50)
]
print(filtered_customers)


   Customer ID  Age Gender  Total Spent  Converted  \
1            2   35      M          600          1   

                        Comments Purchased Category  Stock Region  \
1  Needs improvement in delivery             Beauty     86   West   

  Income Level  Discount Availed  Customer Tenure (Years)     Campaign  \
1          Low                61                       12  Summer Sale   

   Return Rate (%)  Website Visit Frequency  Average Cart Value  
1         1.833412                        6              315.52  


In [8]:
age_bins = pd.cut(df['Age'], bins=[0, 20, 30, 40, 50, 60, 70], labels=['0-20', '21-30', '31-40', '41-50', '51-60', '61-70'])
binned_analysis = df.groupby(age_bins).agg(
    Total_Conversions=('Converted', 'sum'),
    Avg_Cart_Value=('Average Cart Value', 'mean')
)
print(binned_analysis)


       Total_Conversions  Avg_Cart_Value
Age                                     
0-20                   0             NaN
21-30                  5      260.951667
31-40                  6      303.571429
41-50                  1      251.116667
51-60                  0      222.580000
61-70                  0             NaN


<ipython-input-8-cb630e9e2ce4>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_analysis = df.groupby(age_bins).agg(


In [9]:
conversion_matrix = df.pivot_table(values='Converted', index='Gender', columns='Region', aggfunc='mean')
print(conversion_matrix)


Region      East  North  South      West
Gender                                  
F       0.833333    0.5    NaN  0.333333
M       0.000000    1.0    1.0  0.666667


In [10]:
def segment_customer(row):
    if row['Total Spent'] > 500 and row['Customer Tenure (Years)'] > 10:
        return 'High Value'
    elif row['Total Spent'] > 300:
        return 'Medium Value'
    else:
        return 'Low Value'

df['Customer Segment'] = df.apply(segment_customer, axis=1)
print(df[['Customer ID', 'Total Spent', 'Customer Tenure (Years)', 'Customer Segment']])


    Customer ID  Total Spent  Customer Tenure (Years) Customer Segment
0             1          150                        4        Low Value
1             2          600                       12       High Value
2             3          800                       12       High Value
3             4          200                        7        Low Value
4             5          300                       12        Low Value
5             6          450                       13     Medium Value
6             7          670                        8     Medium Value
7             8          120                        3        Low Value
8             9          500                       14     Medium Value
9            10          750                        1     Medium Value
10           11          610                        4     Medium Value
11           12          290                        2        Low Value
12           13          400                        8     Medium Value
13    

In [11]:
import numpy as np

df['Purchase Date'] = pd.to_datetime(
    np.random.choice(pd.date_range('2022-01-01', '2024-12-01'), size=len(df))
)
monthly_trends = df.groupby(df['Purchase Date'].dt.to_period('M'))['Total Spent'].sum()
print(monthly_trends)


Purchase Date
2022-01     290
2022-02     700
2022-03     150
2022-04     510
2022-08    2050
2022-09     750
2022-11     400
2022-12     320
2023-02     400
2023-03     670
2023-07     600
2023-12     120
2024-06     800
2024-07     810
2024-08     350
Freq: M, Name: Total Spent, dtype: int64


In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Total Spent Normalized', 'Discount Availed Normalized']] = scaler.fit_transform(df[['Total Spent', 'Discount Availed']])
print(df[['Total Spent', 'Total Spent Normalized', 'Discount Availed', 'Discount Availed Normalized']])


    Total Spent  Total Spent Normalized  Discount Availed  \
0           150                0.044118                46   
1           600                0.705882                61   
2           800                1.000000                50   
3           200                0.117647                54   
4           300                0.264706                63   
5           450                0.485294                 2   
6           670                0.808824                50   
7           120                0.000000                 6   
8           500                0.558824                20   
9           750                0.926471                72   
10          610                0.720588                38   
11          290                0.250000                17   
12          400                0.411765                 3   
13          320                0.294118                88   
14          510                0.573529                59   
15          350         